In [ ]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Bidirectional, Dropout
from sklearn.model_selection import train_test_split

train_file_path = "train.csv"
test_file_path = "test.csv"
train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)

train_df = train_df[['cleaned_tweet', 'subtask_a']].dropna()
test_df = test_df[['cleaned_tweet', 'subtask_a']].dropna()

train_df['label'] = train_df['subtask_a'].apply(lambda x: 1 if x == 'OFF' else 0)
test_df['label'] = test_df['subtask_a'].apply(lambda x: 1 if x == 'OFF' else 0)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['cleaned_tweet'])
X_train = tokenizer.texts_to_sequences(train_df['cleaned_tweet'])
X_test = tokenizer.texts_to_sequences(test_df['cleaned_tweet'])

max_length = max(len(seq) for seq in X_train)
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')

y_train = np.array(train_df['label'])
y_test = np.array(test_df['label'])

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100  
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length, trainable=True),
    SpatialDropout1D(0.3),
    Bidirectional(LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)),
    Bidirectional(LSTM(64, dropout=0.3, recurrent_dropout=0.3)),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

model.save("safespeak_model.h5")




Epoch 1/50
206/206 [==============================] - 109s 490ms/step - loss: 0.5862 - accuracy: 0.7034 - val_loss: 0.4552 - val_accuracy: 0.8070
Epoch 2/50
206/206 [==============================] - 112s 546ms/step - loss: 0.4040 - accuracy: 0.8255 - val_loss: 0.4688 - val_accuracy: 0.8047
Epoch 3/50
206/206 [==============================] - 110s 534ms/step - loss: 0.2816 - accuracy: 0.8865 - val_loss: 0.5741 - val_accuracy: 0.7849
Epoch 4/50
206/206 [==============================] - 108s 525ms/step - loss: 0.2021 - accuracy: 0.9265 - val_loss: 0.6729 - val_accuracy: 0.7733
Epoch 5/50
206/206 [==============================] - 100s 486ms/step - loss: 0.1516 - accuracy: 0.9470 - val_loss: 0.8366 - val_accuracy: 0.7733
Epoch 6/50
206/206 [==============================] - 113s 550ms/step - loss: 0.1157 - accuracy: 0.9601 - val_loss: 0.8946 - val_accuracy: 0.7709
Epoch 7/50
206/206 [==============================] - 127s 618ms/step - loss: 0.0956 - accuracy: 0.9680 - val_loss: 0.9894 -

In [ ]:
def predict_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post')
    prediction = model.predict(padded_sequence)[0][0]
    return "Harmful" if prediction >= 0.5 else "Safe"

test_text = "You are black and ugly. Move out"
print(f"Prediction: {predict_text(test_text)}")

Prediction: Harmful
